In [1]:
import os
#os.chdir("/content/drive/My Drive/Projects/Face_recog")
os.listdir('.')

['.ipynb_checkpoints',
 'emotion_recog.ipynb',
 'fer.h5',
 'fer.json',
 'fer2013.csv',
 'haarcascade_frontalface_default.xml',
 'openCV.ipynb',
 'video_tester.ipynb',
 'video_tester.py']

In [2]:
import sys, os
import pandas as pd
import numpy as np

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from keras.utils import np_utils
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

In [4]:
df=pd.read_csv('fer2013.csv')

In [5]:
# print(df.info())
# print(df["Usage"].value_counts())

# print(df.head())
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


In [6]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 50
width, height = 48, 48


X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

#cannot produce
#normalizing data between oand 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [7]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

In [8]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)


Epoch 1/50
449/449 [==============================] - 317s 707ms/step - loss: 1.7434 - accuracy: 0.2849 - val_loss: 1.6047 - val_accuracy: 0.3594
Epoch 2/50
449/449 [==============================] - 312s 694ms/step - loss: 1.5299 - accuracy: 0.4007 - val_loss: 1.4151 - val_accuracy: 0.4556
Epoch 3/50
449/449 [==============================] - 310s 691ms/step - loss: 1.4173 - accuracy: 0.4460 - val_loss: 1.3193 - val_accuracy: 0.4890
Epoch 4/50
449/449 [==============================] - 310s 691ms/step - loss: 1.3448 - accuracy: 0.4789 - val_loss: 1.2815 - val_accuracy: 0.5118
Epoch 5/50
449/449 [==============================] - 321s 715ms/step - loss: 1.2987 - accuracy: 0.5019 - val_loss: 1.2702 - val_accuracy: 0.5160
Epoch 6/50
449/449 [==============================] - 311s 692ms/step - loss: 1.2580 - accuracy: 0.5152 - val_loss: 1.2308 - val_accuracy: 0.5266
Epoch 7/50
449/449 [==============================] - 316s 704ms/step - loss: 1.2292 - accuracy: 0.5280 - val_loss: 1.1931 -

In [ ]:
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")